# 2.7.2: Glucose (Solving IVPs)

---

<br>

*Modeling and Simulation in Python*

Copyright 2021 Allen Downey, (License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/))

Revised, Mike Augspurger (2021-present)

<br>

---

In [ ]:
#@title
# Import necessary libraries
from os.path import basename, exists
from os import mkdir

def download(url,folder):
    filename = folder + basename(url)
    if not exists(folder):
        mkdir(folder)
    # fetches the file at the given url if it is not already present
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

download('https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Notebooks/'
        + 'ModSimPy_Functions/modsim.py', 'ModSimPy_Functions/')

from ModSimPy_Functions.modsim import *
import pandas as pd
import numpy as np

In this notebook, we'll implement the minimal model in two ways:

* We'll start by rewriting the differential equations as difference equations; then we'll solve the difference equations using a version of `run_simulation` similar to what we have used in previous chapters.

* Then we'll use a new SciPy function, called `solve_ivp`, to solve the differential equation using a better algorithm.

We'll see that `solve_ivp` is faster and more accurate than `run_simulation`.
As a result, we will use it for the models in the rest of the book.   

First, though, we'll download the glucose and insulin data:

In [ ]:

filename = 'https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Data/glucose_insulin.csv'
# If you are using this notebook offline, you will need to upload this data
# from the Images_and_Data folder.  Comment out the line above, and uncomment the
# line below this one, and run this cell
# filename = '../Images_and_Data/Data/glucose_insulin.csv'

data = pd.read_csv(filename, index_col='time')

## Using a Time Step Loop

### Making a System and a State

Here's a version of `make_system` that takes `params` and `data` as parameters.  Notice that we are setting the base values of glucose and insulin here by assuming that the patient is at these base levels just before the injection.

In [ ]:
def make_system(G0, k1, k2, k3, dt, data):
    t_0 = data.index[0]
    t_end = data.index[-1]
    
    # Assuming that the patient levels are at equilibrium
    # before the injection
    Gb = data.glucose[t_0]
    Ib = data.insulin[t_0]
    
    # Creating the function I to evaluate insulin at any time step
    # The insulin level as a function of time is an input into this
    # model, rather than a variable like G and X
    I = interp1d(data.insulin.index,data.insulin.values)

    # The system needs an attribute called 'init' because we'll
    # use that with "solve_ivp" later in the notebook
    state = pd.Series(dict(G=G0, X=0),dtype=np.float64)
    system = dict(init=state,
                  k1=k1,k2=k2,
                  k3=k3,dt=dt,
                  Gb=Gb, Ib=Ib, I=I,
                  t_0=t_0, t_end=t_end)
    
    return system, state


`make_system` gets `t_0` and `t_end` from the data, and it uses the parameter `G0` as the initial value for `G`. Then it 
returns both a state object (a `Series`) and a system object (a `dictionary`).   We'll make some assumptions here to set the parameters:

In [ ]:
G0 = 270
k1 = 0.02
k2 = 0.02
k3 = 1.5e-05
dt = 2
system, state = make_system(G0, k1, k2, k3, dt, data)


### Defining a Change Function

The minimal model is expressed in terms of differential equations:

<br>

$$\frac{dG}{dt} = -k_1 \left[ G(t) - G_b \right] - X(t) G(t)$$

<br>

$$\frac{dX}{dt} = k_3 \left[I(t) - I_b \right] - k_2 X(t)$$ 

<br>

To simulate this system, we will rewrite them as difference equations. 
If we multiply both sides by $dt$, as we have done before, we have:

<br>

$$dG = \left[ -k_1 \left[ G(t) - G_b \right] - X(t) G(t) \right] dt$$

<br>

$$dX = \left[ k_3 \left[I(t) - I_b \right] - k_2 X(t) \right] dt$$ 

<br>

If we think of $dt$ as a small step in time, these equations tell us how to compute the corresponding changes in $G$ and $X$.  Here's an change function that computes these changes:

In [ ]:
def change_func(t, state, system):
    
    G, X = state
    k1, k2, k3, dt = system['k1'],system['k2'], system['k3'], system['dt']
    I, Ib, Gb = system['I'], system['Ib'], system['Gb']
    
    dGdt = -k1 * (G - Gb) - X*G
    dXdt = k3 * (I(t) - Ib) - k2 * X

    state.G += dGdt * dt
    state.X += dXdt * dt

    return state

We've seen code like this before: this function returns the updated state object return value with the new values of `G` and `X`.  Before running the simulation, it is a good idea to run the change function with the initial conditions:

In [ ]:
change_func(system['t_0'], state, system)

If it runs without errors and there is nothing obviously wrong with the results, we are ready to run the simulation. 

✅ Active reading: Add inline documentation to the change_func above.  Add one line of documentation for each section of code.

### Simulating Glucose Levels

We'll use the following version of `run_simulation`:

In [ ]:
def run_simulation(system, state, change_func):  
    t_array = np.arange(system['t_0'], system['t_end']+1, system['dt'])
    n = len(t_array)
    
    frame = pd.DataFrame(index=t_array, 
                      columns=state.index,
                        dtype=np.float64)
    frame.iloc[0] = state
    
    for i in range(n-1):
        t = t_array[i]
        frame.iloc[i+1] = change_func(t, state, system)
    
    return frame

Notice that we are storing the state ($G$ and $X$) at each time step with a `DataFrame`, rather than with two `Series`. When we make the `DataFrame`, we use `index` to indicate that the index is the array of time stamps, `t_array`, and `columns` to indicate that the column names are the state variables we get from index of the state object.

<br>

We can run it like this, and it returns a `DataFrame` with a row for each time step and a column for each of the state variables, `G` and `X`.

In [ ]:
system, state = make_system(G0, k1, k2, k3, dt, data)
results = run_simulation(system, state,change_func)
results.head()

The following plot shows the simulated glucose levels from the model along with the measured data. 

In [ ]:
data.glucose.plot(style='o', alpha=0.5, label='glucose data',legend=True)
results.G.plot(style='-', color='C0', label='simulation',legend=True,
               xlabel='Time (min)',
               ylabel='Concentration (mg/dL)');

With the parameters we chose, the model fits the data well except during the first few minutes after the injection.
But we don't expect the model to do well in this part of the time series.  The problem is that the model is *non-spatial*; that is, it does not
take into account different concentrations in different parts of the
body. Instead, it assumes that the concentrations of glucose and insulin in blood, and insulin in tissue fluid, are the same throughout the body. This way of representing the body is known among experts as the "bag of blood" model.

<br>

Immediately after injection, it takes time for the injected glucose to
circulate. During that time, we don't expect a non-spatial model to be
accurate. For this reason, we should not take the estimated value of `G0` too seriously; it is useful for fitting the model, but not meant to correspond to a physical, measurable quantity.

✅  Active reading: The rate equation for glucose is:

$$\frac{dG}{dt} = -k_1 \left[ G(t) - G_b \right] - X(t) G(t)$$

Remembering that all the $k$ constants are positive, which of the right hand side terms (or both?) is driving the glucose down in the first part of the model?  Notice that around $t=75$, the glucose level goes below the initial value $G_b$: which of the terms can be responsible for making this happen?

✅ ✅  Answer here

### Simulating the Level of Insulin in the Tissue

Remember that $I$ represents the insulin levels in the blood, while $X$ represents the levels in the tissue fluid, where it actually does the work of reducing glucose.  The following plot shows simulated insulin levels $X$ in the tissue, which is in unspecified units:

In [ ]:
results.X.plot(color='C1', label='insulin in tissue fluid',
               xlabel='Time (min)', 
               ylabel='Concentration (arbitrary units)',
               legend=True);

Remember that `X` represents the concentration of insulin in the "remote compartment", which is believed to be tissue fluid, so we can't compare it to the measured concentration of insulin in the blood.  `X` rises quickly after the initial injection and then declines as the concentration of glucose declines.  Qualitatively, this behavior is as expected, but because `X` is not an observable quantity, we can't validate this part of the model quantitatively.

✅ Active reading: The equation for $X$ is:

$$\frac{dX}{dt} = k_3 \left[I(t) - I_b \right] - k_2 X(t)$$ 

Which term is responsble for driving the rapid increase in $X$ in the first part of the simulation?

✅ ✅  Answer here

## Using a Solver to Solve Differential Equations

To implement the minimal model, we rewrote the differential equations as difference equations with a finite time step, `dt`.
When $dt$ is very small, or more precisely *infinitesimal*, the difference equations are the same as the differential equations.
But in our simulations, $dt$ is 2 min, which is not very small, and definitely not infinitesimal. 

<br>

In effect, the simulations assume that the derivatives $dG/dt$ and $dX/dt$ are constant during each 2 min time step.
This method, evaluating derivatives at discrete time steps and assuming that they are constant in between, is called *Euler's method*.  Euler's method is good enough for many problems, but sometimes it is not very accurate.
In that case, we can usually make it more accurate by decreasing the size of `dt`.
But then it is not very efficient.

<br>

There are other methods that are more accurate and more efficient than Euler's method.
SciPy provides several of them wrapped in a function called `solve_ivp`.
The `ivp` stands for *initial value problem*, which is the term for problems like the ones we've been solving, where we are given the initial conditions and try to predict what will happen.

### Introducing a Slope Function

The ModSim library provides a function called `run_solve_ivp` that makes `solve_ivp` a little easier to use.  To use it, we have to provide a *slope function*, which is similar to an change function, except that uses the terminology of differential equations.  The slope function tell us how much the value of variable is changing at an instant in time.  It takes the same parameters as a change function, too: a time stamp, a state object, and a system object.

<br>

Compare the `change_func` we wrote earlier to our new `slope_func`, which evaluates the differential equations of the minimal model.

In [ ]:
def change_func(t, state, system):
    G, X = state
    k1, k2, k3, dt = system['k1'],system['k2'], system['k3'], system['dt']
    I, Ib, Gb = system['I'], system['Ib'], system['Gb']
    
    dGdt = -k1 * (G - Gb) - X*G
    dXdt = k3 * (I(t) - Ib) - k2 * X

    state.G += dGdt * dt
    state.X += dXdt * dt

    return state

def slope_func(t, state, system):
    G, X = state
    k1, k2, k3, dt = system['k1'],system['k2'], system['k3'], system['dt']
    I, Ib, Gb = system['I'], system['Ib'], system['Gb']
        
    dGdt = -k1 * (G - Gb) - X*G
    dXdt = k3 * (I(t) - Ib) - k2 * X
    
    return dGdt, dXdt

`slope_func` is a little simpler than `change_func` because it only compute the derivatives, that is, the slopes. It doesn't do the updates; the solver does them for us.  **Notice that the slope terms that it returns are in the same order as the state variables (i.e G and dGdt both come first, X and dXdt come second).**  This order is important: it is the way that the solver keeps track of the changes to the state variables.

### Using the Solver

Now we can call `run_solve_ivp` like this:

In [ ]:
system, state = make_system(G0, k1, k2, k3, dt, data)
results2, details = run_solve_ivp(system, slope_func,t_eval=results.index)

Notice how we call `run_solve_ivp`:

<br>

* Like `run_simulation`, it takes a `System` object and a slope function as parameters.

* The `t_eval` option tells the solver at which time points to evaluate a solution: this is optional and is called a "keyword argument", because you need to set the argument equal to the keyword `t_eval` if you want to use it.

* It returns two values: a `DataFrame` ofr results, which we assign to `results2`, and an `OdeResult` object, which we assign to `details`.

<br>

The `OdeResult` object contains information about how the solver ran, including a success code and a diagnostic message.  Make sure that you include two variable names on the left hand side when you run `run_solve_ivp`!

In [ ]:
details.success

In [ ]:
details.message

It's important to check these messages after running the solver, in case anything went wrong.

<br>

The `DataFrame` has one row for each time step and one column for each state variable. In this example, the rows are time from 0 to 182 minutes; the columns are the state variables, `G` and `X`.  Here are the first few time steps:

In [ ]:
results2.head()

Because we used `t_eval=results.index`, the time stamps in `results2` are the same as in `results`, which makes them easier to compare.  The following figure shows the results from `run_solve_ivp` along with the results from `run_simulation`:

In [ ]:
results.G.plot(style='--', label='simulation',legend=True)
results2.G.plot(style='-', label='solve ivp',xlabel='Time (min)',
         ylabel='Concentration (mg/dL)',legend=True);

The differences are barely visible.
We can compute the relative differences like this:



In [ ]:
diff = results.G - results2.G
percent_diff = diff / results2.G * 100

And we can use `describe` to compute summary statistics:

In [ ]:
percent_diff.abs().describe()


The mean relative difference is about 0.65% and the maximum is a little more than 1%.
Here are the results for `X`.

In [ ]:
results.X.plot(style='--', label='simulation',legend=True)
results2.X.plot(style='-', label='solve ivp',xlabel='Time (min)', 
         ylabel='Concentration (arbitrary units)',legend=True);

These differences are little bigger, especially at the beginning.  If we use `run_simulation` with smaller time steps, the results are more accurate, but they take longer to compute.  For some problems, we can find a value of `dt` that produces accurate results in a reasonable time. However, if `dt` is *too* small, the results can be inaccurate again. So it can be tricky to get it right.

<br>

The advantage of `run_solve_ivp` is that it chooses the step size automatically in order to balance accuracy and efficiency.  You can use keyword arguments to adjust this balance, but most of the time the results are accurate enough, and the computation is fast enough, without any intervention.

In [ ]:
diff = results.X - results2.X
percent_diff = diff / (results2.X.mean()) * 100
percent_diff.abs().describe()

---

## Summary

In this notebook, we implemented the glucose minimal model two ways, using `run_simulation` and `run_solve_ivp`, and compared the results.
We found that in this example, `run_simulation`, which uses Euler's method, is probably good enough.

<br>

---

### Exercise 1

Our solution to the differential equations is only approximate because we used a finite step size, `dt=2` minutes.
If we make the step size smaller, we expect the solution to be more accurate.  Run the simulation with `dt=1` and compare the results for `G` with the `dt = 2` results. 

In [ ]:
# Run simulation with dt = 1 and dt = 2
# Save the results as results1 and results2




In [ ]:
#Plot both results with the glucose data
data.glucose.plot(style='o', alpha=0.5, label='glucose data',legend=True)
results1.G.plot(style='--', color='C1', label='dt = 1',legend=True)
results2.G.plot(style='-', color='C3', label='dt = 2',
                xlabel='Time (min)', 
                ylabel='Concentration (mg/dL)',
                ylim=[80,270],legend=True);

### Exercise 2

Comment on the results of your comparison.  Do the results differ significantly between a dt = 1 and dt = 2?   Now try larger dt values: do we still get usable results with a dt = 5? 10? 20?

<br>

Why might it be useful to have a larger dt even if it costs us a little accuracy?

✅ ✅  Answer here.